In [44]:
import functools
print = functools.partial(print, flush=True)
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import numpy as np
import datetime
import time, math
from Encoder import Encoder
from Decoder import Decoder
from Hyperparameters import args
import argparse
import pandas as pd
from datetime import date

import argparse
import pandas as pd
from datetime import date


import torch
from torch import nn
import gensim
# import fasttext
from gensim.models import FastText
import random
import operator
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pickle
import joblib
from Hyperparameters import args
import nltk

from textdataMimic import TextDataMimic

In [4]:
textData = TextDataMimic("mimic", "../clinicalBERT/data/", "discharge", trainLM=False, test_phase=False, big_emb = False, new_emb = True)
        # self.start_token = self.textData.word2index['START_TOKEN']
        # self.end_token = self.textData.word2index['END_TOKEN']
args['vocabularySize'] = textData.getVocabularySize()
args['chargenum'] = 2
args['embeddingSize'] = textData.index2vector.shape[1]


self.new emb in textData is:  True
self.big emb in textData is:  False
using new embeddings
using this embedding model:./data/mimic3/new_mimic_word2vec_200.model 
./data/mimic3//mimic3_processed_new200_discharge.pkl
Found already saved data at ./data/mimic3//mimic3_processed_new200_discharge.pkl! Loading that instead
Loading dataset from ./data/mimic3//mimic3_processed_new200_discharge.pkl
training: 	 26245
dev: 	 3037
testing: 	 3063
w2i shape:  337758
i2w shape:  337758
embeding shape:  (337758, 200)
loaded
set
Loaded mimic: 337758 words, 0 QA


In [47]:
# train_X = textData.datasets["train"]
# print(train_X.shape)
# pd.DataFrame(train_X).head()

In [41]:
class Vocabulary:
    """A vocabulary, assigns IDs to tokens"""

    def __init__(self, w2i, i2w):
        self.w2i = w2i
        self.i2w = i2w


class LSTM_Model(nn.Module):
    """
        LSTM
    """

    def __init__(self, w2i, i2w, LM, i2v=None, dimension = 200):
        """
        Args:
            args: parameters of the model
            textData: the dataset object
        """
        super(LSTM_Model, self).__init__()
        
        self.word2index = w2i
        self.index2word = i2w
        self.max_length = 512


        # TODO try using the language model embedding after training!
#         self.embedding = LM.embedding
        # self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(i2v))

        self.embedding = nn.Embedding(textData.getVocabularySize(), 200)
        self.dimension = dimension
        self.lstm = nn.LSTM(input_size=200,
                            hidden_size=self.dimension,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)
        self.drop = nn.Dropout(p=0.5)

        self.fc = nn.Linear(2*dimension, 1)

    def forward(self, text, text_len):

        
        
#         self.encoderInputs = encoderInputs.to(args['device'])
#         self.encoder_lengths = encoder_lengths
        
        text_emb = self.embedding(text)
        text_len = text_len.cpu()

        packed_input = pack_padded_sequence(text_emb, text_len, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        out_forward = output[range(len(output)), text_len - 1, :self.dimension]
        out_reverse = output[:, 0, self.dimension:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)
        text_fea = self.drop(out_reduced)

        text_fea = self.fc(text_fea)
        text_fea = torch.squeeze(text_fea, 1)
        text_out = torch.sigmoid(text_fea)

        return text_out

In [22]:
args

{'test': None,
 'createDataset': True,
 'playDataset': 10,
 'reset': True,
 'device': 'cuda',
 'rootDir': './artifacts/',
 'retrain_model': 'No',
 'maxLength': 1000,
 'vocabularySize': 337758,
 'hiddenSize': 200,
 'numLayers': 2,
 'initEmbeddings': True,
 'embeddingSize': 200,
 'capsuleSize': 50,
 'numEpochs': 1,
 'saveEvery': 2000,
 'batchSize': 256,
 'learningRate': 0.001,
 'dropout': 0.9,
 'clip': 5.0,
 'encunit': 'lstm',
 'decunit': 'lstm',
 'enc_numlayer': 2,
 'dec_numlayer': 2,
 'maxLengthEnco': 1000,
 'maxLengthDeco': 1001,
 'temperature': 1.0,
 'classify_type': 'multi',
 'task': 'charge',
 'scheduler': 'multistep',
 'lr_decay': 0.97,
 'patience': 5,
 'threshold': 0.0001,
 'cooldown': 0,
 'min_lr': 5e-05,
 'milestones': [25, 50, 75],
 'chargenum': 2}

In [48]:
# Training Function
destination_folder = "./results/"
device = args["device"]
args["batchSize"] = 128
eval_every = 50
def train(model,
          optimizer,
          criterion = nn.BCELoss(),
          num_epochs = 5,          
          file_path = destination_folder,
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []
    
    batches = textData.getBatches()

    # training loop
    model.train()
    for epoch in range(num_epochs):


        for index, batch in enumerate(batches):
            x = {}
            x['enc_input'] = autograd.Variable(torch.LongTensor(batch.encoderSeqs)).to(args['device'])
            x['enc_len'] = torch.LongTensor(batch.encoder_lens).to(args['device'])
            x['labels'] = autograd.Variable(torch.FloatTensor(batch.label)).to(args['device'])
            
            output = model(x['enc_input'],x['enc_len'])
            
#             print(output)

#             print(output.shape)
#             print(x['labels'].shape)

            loss = criterion(output, x['labels'])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                
                model.eval()
                with torch.no_grad():
                    batches = textData.getBatches("dev")
                    
#                   # validation loop     
                    for index, batch in enumerate(batches):
                        x = {}
                        x['enc_input'] = autograd.Variable(torch.LongTensor(batch.encoderSeqs)).to(args['device'])
                        x['enc_len'] = torch.LongTensor(batch.encoder_lens).to(args['device'])
                        x['labels'] = autograd.Variable(torch.FloatTensor(batch.label)).to(args['device'])
            
                        output = model(x['enc_input'],x['enc_len'])
                        loss = criterion(output, x['labels'])
                        valid_running_loss += loss.item()

#                 # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
#                 valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

#                 # resetting running values
                running_loss = 0.0                
#                 valid_running_loss = 0.0
#                 model.train()

                # print progress
    
                print(f"Epoch number: {epoch+1}. Epoch loss: {average_train_loss}")
#                 print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
#                       .format(epoch+1, num_epochs, global_step, num_epochs*len(),
#                               average_train_loss, average_valid_loss))
                
                # checkpoint
#                 if best_valid_loss > average_valid_loss:
#                     best_valid_loss = average_valid_loss
#                     save_checkpoint(file_path + '/model.pt', model, optimizer, best_valid_loss)
#                     save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
#     save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
#     print('Finished Training!')


model = LSTM_Model(textData.word2index, textData.index2word, LM = None,i2v = None).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model=model, optimizer=optimizer, num_epochs=5)

RuntimeError: CUDA out of memory. Tried to allocate 258.00 MiB (GPU 0; 4.00 GiB total capacity; 2.66 GiB already allocated; 0 bytes free; 2.95 GiB reserved in total by PyTorch)